# ❇️VECTORIZERS #

##  ` 0️⃣Opis zadania` ### 


## 🇵🇱 Zadanie 2.2.8** (opcjonalnie)

Nowy zbiór z `KLEJ` - [PolEmo2.0-IN](https://clarin-pl.eu/dspace/handle/11321/710). To zestaw recenzji online z dziedziny medycyny i hoteli. Zadaniem jest przewidzenie sentymentu recenzji.

* input: `../input/klej/klej_polemo2.0-in/train.tsv`
* models: `../models/word2vec/klej_polemo2.0-in_train`

##  ` 1️⃣Import bibliotek` ### 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter, defaultdict
import spacy

from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import random
import os

from gensim.utils import simple_preprocess
from tensorflow.keras.utils import to_categorical

import mlflow
from itertools import product
import time
import datetime
import warnings
import tensorflow as tf

import team_helper as th
from team_helper import recall, precision, f1, get_y
from team_helper import random_polemo2_opinion as random_opinion

from sklearn.model_selection import cross_validate
from scikitplot.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score

In [15]:
from scikitplot.estimators import plot_learning_curve

##  ` 3️⃣Usunięcie niepotrzebnych ostrzeżeń + ustawienie mlflow` ### 

In [2]:
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [20]:
mlflow.set_tracking_uri("file:///home/jovyan/nlp2/shared-mlruns/team-three/mariusz")

##  ` 4️⃣Próba zapewnienia powtarzalności eksperymentu` ### 

In [3]:
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
tf.compat.v1.set_random_seed(seed_value)

##  ` 5️⃣Eksploracja zbioru` ### 

In [4]:
df = pd.read_csv('../input/klej/klej_polemo2.0-in/train.tsv', sep='\t')

In [5]:
df

,sentence,target
0,Super lekarz i człowiek przez duże C . Bardzo ...,__label__meta_plus_m
1,Bardzo olewcze podejscie do pacjenta . Przypro...,__label__meta_minus_m
2,Lekarz zalecił mi kurację alternatywną do doty...,__label__meta_amb
3,Konsumenci oczywiście kierują się ceną . Te l...,__label__meta_zero
4,Pani Doktor Iwona jest profesjonalistką w każd...,__label__meta_plus_m
...,...,...
5739,"Centralne skrzydło jest jednokondygnacyjne , z...",__label__meta_zero
5740,Ogólnie w hotelu panuje balagan informacyjny -...,__label__meta_minus_m
5741,Przybyli śmy z rodziną na krotki wypoczynek . ...,__label__meta_amb
5742,Opinię może wyrazić dzisiaj każdy i jest to je...,__label__meta_zero


In [6]:
df['target'].value_counts(normalize=True)

__label__meta_minus_m    0.380223
__label__meta_plus_m     0.270369
__label__meta_amb        0.181929
__label__meta_zero       0.167479
Name: target, dtype: float64

##  ` 6️⃣Wyodrębnienie zmiennych i preprocessing` ### 

In [7]:
X_oryg = df['sentence'].map(th.simple_tokens).apply(lambda x:' '.join(x))
Xprep = df['sentence'].map(th.preprocessing).map(simple_preprocess).apply(lambda x:' '.join(x))
y = df['target'].factorize()[0]

In [8]:
def lemmatize_text(doc):
    return ' '.join([token.lemma_ for token in th.nlp_sm(doc)])
Xprep = Xprep.apply(lemmatize_text)

In [9]:
import texthero as hero
df["clean_text"] = hero.clean(df["sentence"])

In [10]:
SELECTED_X = df['clean_text']

In [11]:
SELECTED_X = Xprep

In [12]:
EXP_NAME = 'Vectorizers_polemo2_0_IN_X_prep_t'
SELECTED_X = Xprep

# 
max_features_list = [300,500, 1000] 

# 
tokenizer_list = [th.polish_tokenizer_md]

# 
stop_words_list = [th.get_stopwords(th.spacy_stop_words_md),[]]
#stop_words_list = [[]]

# 
vectorizers_list = [CountVectorizer, TfidfVectorizer] 

min_df_list = [0.01, 0.1, 1]

max_df_list = [0.3, 0.5, 1.0]

models_list = th.get_models(use_dummy=False)

##  ` 7️⃣Eksperyment` ### 

In [ ]:
params_cnt = len(max_features_list)*\
len(tokenizer_list)*\
len(stop_words_list)*\
len(vectorizers_list)*\
len(min_df_list)*len(max_df_list)

exp_start = datetime.datetime.now()

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0) 

for n,(Vectorizer, max_features, tokenizer, stop_words, min_df, max_df) in \
enumerate(product(vectorizers_list, max_features_list, tokenizer_list, stop_words_list, min_df_list, max_df_list),1):
    
    model_str = f'max_features = {max_features} tokenizer={tokenizer.__name__}, vectorizer={Vectorizer.__name__}, stop_words = {len(stop_words)}, dfs=min:{min_df}, max:{max_df}'
    
    print(f'{n}/{params_cnt}')
    print(model_str)
    
    kwargs = {'scoring': ['f1_micro']}
    
    
    vectorizer_kwargs = {'max_features': max_features,
                         'tokenizer': tokenizer,
                         'stop_words': stop_words,
                         'min_df':min_df,
                         'max_df':max_df}
    vec = Vectorizer(**vectorizer_kwargs)

    start = time.time()
    X = vec.fit_transform(SELECTED_X).toarray()
    end = time.time()
    print('##'*10)
    print(f'Vectorization time: {end-start}')
    print('##'*10)

    for model_name, model_obj in models_list:
        with mlflow.start_run(experiment_id=th._eid(EXP_NAME), run_name=f'{model_str}-{model_name}'):
            start = time.time()
            scores = cross_validate(model_obj, X, y, cv=cv, scoring=kwargs['scoring'], return_train_score=True)    
            #print(scores)
            mean = np.around( np.mean(scores['test_f1_micro']), 2)
            std = np.around(np.std(scores['test_f1_micro']),2)

            print(f'model = {model_name} score = {mean}')

            y_pred = cross_val_predict(model_obj, X, y, cv=cv)

            #fig = plt.subplots(figsize=(10, 10))
            fig = plt.figure()
            fig = plot_confusion_matrix(y, y_pred, title='model: {}'.format(model_name))
            plt.savefig("confusion_matrix.png")
            
            fig = plt.figure()
            fig = plot_learning_curve(model_obj, X, y, title='model: {}'.format(model_name))
            plt.savefig("learning_curve.png")

            end = time.time()
            print('time: ',round((end-start),2))
            print('-'*10)

            # log params
            mlflow.log_param('model', model_name)
            mlflow.log_param('max_features', max_features)
            mlflow.log_param('tokenizer', tokenizer.__name__)
            mlflow.log_param('vectorizer', Vectorizer.__name__)
            mlflow.log_param('lenstop_words', len(stop_words))
            mlflow.log_param('min_df', min_df)
            mlflow.log_param('max_df', max_df)

            # log metrics
            mlflow.log_metric('mean_train_f1_micro', np.around( np.mean(scores['train_f1_micro']), 2))
            mlflow.log_metric('mean_test_f1_micro', np.around( np.mean(scores['test_f1_micro']), 2))

            mlflow.log_metric('std_train_f1_micro', np.around( np.std(scores['train_f1_micro']), 2))
            mlflow.log_metric('std_test_f1_micro', np.around( np.std(scores['test_f1_micro']), 2))
            
            mlflow.log_metric('calc_time', (end-start))

            # log artifacts
            mlflow.log_artifact("confusion_matrix.png")
            mlflow.log_artifact("learning_curve.png")
            #plt.show()


exp_end = datetime.datetime.now()
print('='*10)
print(f'Start of experiment: {exp_start}')
print(f'End of experiment: {exp_end}')
th.calculate_delta((exp_end-exp_start))

dfruns = mlflow.search_runs(experiment_ids=[th._eid(EXP_NAME)])

1/108
max_features = 300 tokenizer=polish_tokenizer_md, vectorizer=CountVectorizer, stop_words = 381, dfs=min:0.01, max:0.3
####################
Vectorization time: 167.49834942817688
####################
model = decision tree score = 0.48
time:  1.62
----------
model = random forest score = 0.55
time:  6.88
----------
model = extra-trees score = 0.45
time:  6.29
----------
model = lightgbm score = 0.69
time:  8.13
----------
model = catboost score = 0.72
time:  33.27
----------
model = xgboost score = 0.72
time:  45.41
----------
2/108
max_features = 300 tokenizer=polish_tokenizer_md, vectorizer=CountVectorizer, stop_words = 381, dfs=min:0.01, max:0.5


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['cal', 'chcieć', 'czas', 'kierunek', 'mieć', 'możliwy', 'musić', 'móc', 'rok', 'wiedzieć'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


####################
Vectorization time: 155.29947757720947
####################
model = decision tree score = 0.48
time:  1.42
----------
model = random forest score = 0.55
time:  6.34
----------
model = extra-trees score = 0.47
time:  5.38
----------
model = lightgbm score = 0.69
time:  7.38
----------
model = catboost score = 0.72
time:  29.61
----------
model = xgboost score = 0.72
time:  56.84
----------
3/108
max_features = 300 tokenizer=polish_tokenizer_md, vectorizer=CountVectorizer, stop_words = 381, dfs=min:0.01, max:1.0


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['cal', 'chcieć', 'czas', 'kierunek', 'mieć', 'możliwy', 'musić', 'móc', 'rok', 'wiedzieć'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


####################
Vectorization time: 160.43988275527954
####################
model = decision tree score = 0.48
time:  1.54
----------
model = random forest score = 0.55
time:  6.85
----------
model = extra-trees score = 0.45
time:  5.94
----------
model = lightgbm score = 0.7
time:  9.74
----------
model = catboost score = 0.72
time:  33.51
----------
model = xgboost score = 0.72
time:  64.27
----------
4/108
max_features = 300 tokenizer=polish_tokenizer_md, vectorizer=CountVectorizer, stop_words = 381, dfs=min:0.1, max:0.3


...

####################
Vectorization time: 160.47012066841125
####################
model = decision tree score = 0.48
time:  7.94
----------
model = random forest score = 0.53
time:  13.08
----------
model = extra-trees score = 0.45
time:  14.17
----------
model = lightgbm score = 0.7
time:  20.54
----------
model = catboost score = 0.72
time:  143.96
----------
model = xgboost score = 0.73
time:  163.82
----------
98/108
max_features = 1000 tokenizer=polish_tokenizer_md, vectorizer=TfidfVectorizer, stop_words = 381, dfs=min:1, max:0.5


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['cal', 'chcieć', 'czas', 'kierunek', 'mieć', 'możliwy', 'musić', 'móc', 'rok', 'wiedzieć'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


####################
Vectorization time: 155.87742042541504
####################
model = decision tree score = 0.47
time:  7.89
----------
model = random forest score = 0.54
time:  12.33
----------
model = extra-trees score = 0.43
time:  14.15
----------
model = lightgbm score = 0.7
time:  19.57
----------
model = catboost score = 0.72
time:  142.85
----------
model = xgboost score = 0.73
time:  160.98
----------
99/108
max_features = 1000 tokenizer=polish_tokenizer_md, vectorizer=TfidfVectorizer, stop_words = 381, dfs=min:1, max:1.0


/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['cal', 'chcieć', 'czas', 'kierunek', 'mieć', 'możliwy', 'musić', 'móc', 'rok', 'wiedzieć'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


####################
Vectorization time: 151.5137951374054
####################
model = decision tree score = 0.48
time:  8.58
----------
model = random forest score = 0.53
time:  12.98
----------
model = extra-trees score = 0.46
time:  15.13
----------
model = lightgbm score = 0.7
time:  19.89
----------
model = catboost score = 0.73
time:  145.43
----------
model = xgboost score = 0.73
time:  198.56
----------
100/108
max_features = 1000 tokenizer=polish_tokenizer_md, vectorizer=TfidfVectorizer, stop_words = 0, dfs=min:0.01, max:0.3
####################
Vectorization time: 164.6885859966278
####################
model = decision tree score = 0.48
time:  9.56
----------
model = random forest score = 0.53
time:  14.43
----------
model = extra-trees score = 0.44
time:  16.73
----------
model = lightgbm score = 0.71
time:  30.48
----------
model = catboost score = 0.73
time:  197.13
----------
model = xgboost score = 0.74
time:  197.7
----------
101/108
max_features = 1000 tokenizer=polis

In [ ]:
!mlflow ui --backend-store-uri /home/jovyan/nlp2/shared-mlruns/team-three/mariusz --default-artifact-root /home/jovyan/nlp2/shared-mlruns/team-three/mariusz --port 5001

In [19]:
!mlflow ui

[2021-10-24 06:47:24 +0000] [623] [INFO] Starting gunicorn 20.1.0
[2021-10-24 06:47:24 +0000] [623] [INFO] Listening at: http://127.0.0.1:5000 (623)
[2021-10-24 06:47:24 +0000] [623] [INFO] Using worker: sync
[2021-10-24 06:47:24 +0000] [625] [INFO] Booting worker with pid: 625
^C
[2021-10-24 06:48:21 +0000] [623] [INFO] Handling signal: int
[2021-10-24 06:48:21 +0000] [625] [INFO] Worker exiting (pid: 625)
